In [86]:
from sklearn.model_selection import train_test_split
import re
import string
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import warnings
import pickle
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.layers import TextVectorization



warnings.filterwarnings('ignore')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sefineh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
resume_df=pd.read_csv('dataset/resume.csv')
resume_copied=resume_df.copy()
resume_df.head()

In [88]:
stopwords_set = set(stopwords.words('english')+['``',"''"])
def clean_resume_text(resume):
    resume = resume.lower()
    resume = re.sub('http\S+\s*',' ',resume) #to remove url
    resume = ''.join([w for w in resume if not w.isdigit()]) # remove the digits
    resume = re.sub('RT|cc',' ',resume) # to remove RT and cc
    resume = re.sub('#\S+','',resume) # to remove hastags
    resume = re.sub('@\S+',' ',resume) # to remove mentions
    resume = ''.join([w for w in resume if w not in string.punctuation])# to remove puntuations
    resume = re.sub('\W',' ',resume)
    #resume = ''.join([w for w in resume if w not in stopwords_set])
    resume = re.sub(r'[^\x00-\x7f]',r' ',resume)
    resume = re.sub('\s+',' ',resume)# to remove extra spaces
    return resume

In [89]:
resume_df['Resume']=resume_df.Resume.apply(lambda x: clean_resume_text(x))
labelEncoder=LabelEncoder()
temp_category=resume_df['Category'].copy()
resume_df["Category"]=labelEncoder.fit_transform(resume_df["Category"])


In [90]:
with open('dict.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('dict.pickle', 'rb') as handle:
    dict = pickle.load(handle)

dict

{6: 'Data Science',
 12: 'HR',
 0: 'Advocate',
 1: 'Arts',
 24: 'Web Designing',
 16: 'Mechanical Engineer',
 22: 'Sales',
 14: 'Health and fitness',
 5: 'Civil Engineer',
 15: 'Java Developer',
 4: 'Business Analyst',
 21: 'SAP Developer',
 2: 'Automation Testing',
 11: 'Electrical Engineering',
 18: 'Operations Manager',
 20: 'Python Developer',
 8: 'DevOps Engineer',
 17: 'Network Security Engineer',
 19: 'PMO',
 7: 'Database',
 13: 'Hadoop',
 10: 'ETL Developer',
 9: 'DotNet Developer',
 3: 'Blockchain',
 23: 'Testing'}

In [91]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 300

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer.adapt(resume_df["Resume"].values)

In [92]:
# vectorize_layer.get_vocabulary()
train_set,test_set=train_test_split(resume_df,test_size=0.2,random_state=0)

In [93]:
embedding_dim=300
model = keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dense(32,activation='relu'),
  tf.keras.layers.Dense(25,activation='softmax')
])

In [235]:
train=tf.convert_to_tensor(train_set["Resume"].values)
target=tf.convert_to_tensor(train_set["Category"].values)
# val_set=tf.convert_to_tensor(val_set["Resume"].values)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [236]:
model.fit(train,target,epochs=15)

Epoch 1/15
25/25 [==============================] - 1s 21ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 2/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 3/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 4/15
25/25 [==============================] - 0s 18ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 5/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 6/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 7/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 8/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 9/15
25/25 [==============================] - 0s 18ms/step - loss: 12.8733 - accuracy: 1.0000
Epoch 10/15
25/25 [==============================] - 0s 19ms/step - loss: 12.8733 - accuracy: 1.0000

In [238]:
model.evaluate(tf.convert_to_tensor(test_set["Resume"]),tf.convert_to_tensor(test_set["Category"]))

7/7 [==============================] - 0s 22ms/step - loss: 13.1581 - accuracy: 0.9948


[13.158133506774902, 0.9948186278343201]

In [237]:
!mkdir -p savedmodel
model.save("savedmodel/model")

In [119]:

new_model=tf.keras.models.load_model("savedmodel/model",\
                                     custom_objects={'vectorize_layer':vectorize_layer,\
                                                    "custom_standardization":custom_standardization})

In [239]:

import textract
text = textract.process("My resume Intern.pdf")
text=clean_resume_text(str(text))
predicted=model.predict(tf.convert_to_tensor([text])).argmax()
print(f"The resume belongs to {dict[predicted]}")


The resume belongs to Web Designing


In [241]:
text=resume_df["Resume"]
text=text.values[0]
predicted=model.predict(tf.convert_to_tensor([text]))

print(predicted.argmax())
print(dict[new_model.predict([text]).argmax()])

6
Data Science


In [242]:
encodings=np.argpartition(predicted[0],-5)[-5:]
encodings=encodings[np.argsort(predicted[0][encodings])]
encodings=reversed(encodings)
index=0
potentials={1:"First",2:"Second",3:"Third",4:"Fourth",5:"Fifth"}
for encoding in encodings:
    index+=1
    print(f"The {potentials[index]} recommended: ==>",dict[encoding])

The First recommended: ==> Data Science
The Second recommended: ==> DevOps Engineer
The Third recommended: ==> Web Designing
The Fourth recommended: ==> Hadoop
The Fifth recommended: ==> DotNet Developer


# Find the most relevant resumes that match the job description.

In [243]:
job_description="""


You will have the opportunity to join an artificial intelligence (AI) company for life sciences, that supports the

industry in bringing the right drug to the right patient at speed.

THE ROLE

By joining an established team of data scientists as an NLP specialist, you will:
Work closely with both data engineers and other data scientists
Help model unstructured data sets
You will be creating and delivering Data Science/NLP projects regularly.
You will be effectively collaborating with colleagues to solve business problems.
Build NLP processing pipelines
Work on conducting proper testing to remove bias
Apply state of the art NLP solutions to solve real-world problems
Your Skills And Experience

To be a fit for this position, you need to have:
Strong knowledge of working with Python, SQL, and Python libraries
Proven industry experience working with NLP tools like BERT, NLTK, GenSim, or similar
Knowledge of Public Clouds, ideally AWS is nice to have
Have a background in computational linguistics, text mining, topic modeling, semantic analysis or text classification, or similar
Fluency in English is a must

"""

In [244]:
#What is the job description about

index=model.predict([job_description]).argmax()
candidate_resume=resume_df[resume_df["Category"]==index]["Resume"]

print("The job description is about: \n\t\t\t",dict[index])

The job description is about: 
			 Mechanical Engineer


In [220]:
#Find the probabilites of a job to be part of each category of the dataset
job_prop=model.predict([job_description])
# Find the probabilites of the candidate resume to each category of the dataset
candidate_prob=model.predict(candidate_resume)
# candidate_prob
job_prop

array([[3.7579564e-04, 2.4757410e-06, 1.1377738e-05, 2.2445786e-04,
        1.1876457e-01, 2.2562090e-01, 2.2589774e-03, 6.6879060e-04,
        1.0431462e-05, 2.2538290e-03, 1.5840247e-04, 1.6940762e-06,
        2.3348026e-07, 4.5474612e-06, 2.3147702e-06, 5.9800595e-07,
        4.3587261e-01, 5.2241944e-07, 8.3095962e-05, 1.6385786e-03,
        5.6193457e-03, 2.4762666e-03, 1.4116225e-01, 4.5851846e-03,
        5.8202773e-02]], dtype=float32)

# I need word embedding model to make a cosine similarity between a job description and the candidate resumes

In [221]:
similarity=cosine_similarity(job_prop,candidate_prob)
similarity_tupled=list(zip(similarity.tolist()[0],candidate_resume.index))
similarity_tupled=sorted(similarity_tupled,key=lambda x:x[0],reverse=True)
top_rated=similarity_tupled[:5]
top_rated

In [208]:
print("The top five rated resume: \n\n")
top_rated_indices=[index[1] for index in top_rated]
resume_copied.iloc[top_rated_indices]

The top five rated resume: 




,Category,Resume
187,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
202,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
212,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
217,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
192,Mechanical Engineer,Education Details \r\nJanuary 2018 Bachelor's ...
